<h1 align=left><font size = 5>Applied Data Science Capstone - The Battle of Neighborhoods</font></h1>

## Introduction
* A JSON file with **New York Neighborhoods** data (including coordinates) is downloaded and imported into a dataframe
* The **Foursquare API** is used to get the most common venue categories in each neighborhood (**explore** function)
* The information is processed to score every neighborhood according to some parameters with preferences for facilities
* The Folium library is used later to visualize the recommended neighborhoods for the input preferences


### Dependencies resolution
Before fetching data and start working with it, let's import all the dependencies that are needed

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('>>> Pandas and Numpy imported')

import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
print('>>> requests and json imported')

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('>>> Geopy installed and geocoders imported')

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('>>> matplotlib and colors imported')

# import k-means from clustering stage
# from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('>>> Folium 0.5.0 isntalled and imported')

print('>>> ALL LIBRARIES IMPORTED!')

>>> Pandas and Numpy imported
>>> requests and json imported
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0         conda-forge
    

### Download the New York Dataset

The NYU - Spatial Data Repository web provides a 2014 New York City Neighborhood Names shapefile for free (https://geo.nyu.edu/catalog/nyu_2451_34572), with information from the New York City Department of City Planning.

This dataset includes a total of 5 boroughs and 306 neighborhoods, as well as the the latitude and longitude coordinates of each one.

As this file was used in previous examples along the course, it can be already downloaded from the Cognitive Class server as a json file. So a simple `wget` command can be used to get it (even when it should already be available in the labs storage, I will include the wget command anyway).

In [2]:
print('Fetching the data file...')
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('>>> Data downloaded!')

Fetching the data file...
>>> Data downloaded!


### Load and check the data

Now, the data is loaded in a Python dictionary

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

The relevant data is in the *features* key, which is basically a list of the neighborhoods.A new variable is defined for it.

In [4]:
neighborhoods_data = newyork_data['features']

Taking a look at the first item in this list, we can see the neighborhood name, the borough it belongs to, and the coordinates

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

### Tranform the list into a *pandas* dataframe

Just transforming this data in dictionary format into a *pandas* dataframe. First, an empty dataframe is created

In [6]:
# Define the dataframe required columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# Instantiate the dataframe, empty yet
neighborhoods = pd.DataFrame(columns=column_names)

Now, looping through the list, filling the dataframe row by row.

In [7]:
for data in neighborhoods_data:
    # Iterating on every item in the list
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
    
    # Coordinates have to be retrieved from the geometry (point) attributes
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    # And all values are appended to the created dataframe
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
print('>>> Dataframe finished!')

>>> Dataframe finished!


Now, inspecting the resulting dataframe, to check first that the columns are right

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And later, to make sure that the dataset has all the information( 5 boroughs and 306 neighborhoods).

In [9]:
print("The 'neighborhoods' dataframe has...")
print('    - {} boroughs'.format(len(neighborhoods['Borough'].unique())))
print('    - {} neighborhoods'.format(neighborhoods.shape[0]))

The 'neighborhoods' dataframe has...
    - 5 boroughs
    - 306 neighborhoods


## Adding the Foursquare information 

The Foursquare API is used to explore the neighborhoods, getting categorized venues for every one

### First, definition of Foursquare Credentials and Version

In [10]:
CLIENT_ID = 'AKKTJPIAVNM4VGQCBL2KV1Q3WVCKFGLGFKDRFOZBGUXCMJ4O' # Foursquare ID
CLIENT_SECRET = 'DBJROEYJR4G1TDQXRTWANGWLWQ1CTDQC5MDMVXH1NLHLF5DF' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Foursquare credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Foursquare credentials:
CLIENT_ID: AKKTJPIAVNM4VGQCBL2KV1Q3WVCKFGLGFKDRFOZBGUXCMJ4O
CLIENT_SECRET:DBJROEYJR4G1TDQXRTWANGWLWQ1CTDQC5MDMVXH1NLHLF5DF


### Now, definition of a *getNearbyVenues* function
The Foursquare API (*explore* endpoint) is called to get a number of venues and categories for a location and radius
* The neighborhood is passed as a parameter, together with the location coordinates
* Maximum radius and number of venues are fixed constants

In [11]:
LIMIT = 100     # limit of number of venues returned by Foursquare API
RADIUS = 500    # define radius

def getNearbyVenues(neighborhood, latitude, longitude, radius=RADIUS, limit=LIMIT):
    
    venues_list=[]
    for name, lat, lng in zip(neighborhood, latitude, longitude):
        print('Fetching venues for ... ' + name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        fs_response = requests.get(url).json()
        print('Code: ' + str(fs_response["meta"]["code"]))
        results = fs_response["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now Calling the above function on each neighborhood and create a new dataframe called *ny_venues*.

In [12]:
# Passing data for every neighborhood (the loop is within the function)
ny_venues = getNearbyVenues(neighborhood=neighborhoods['Neighborhood'],
                                   latitude=neighborhoods['Latitude'],
                                   longitude=neighborhoods['Longitude']
                                  )
print ('>>> ALL VENUES FETCHED!')

Fetching venues for ... Wakefield
Code: 200
Fetching venues for ... Co-op City
Code: 200
Fetching venues for ... Eastchester
Code: 200
Fetching venues for ... Fieldston
Code: 200
Fetching venues for ... Riverdale
Code: 200
Fetching venues for ... Kingsbridge
Code: 200
Fetching venues for ... Marble Hill
Code: 200
Fetching venues for ... Woodlawn
Code: 200
Fetching venues for ... Norwood
Code: 200
Fetching venues for ... Williamsbridge
Code: 200
Fetching venues for ... Baychester
Code: 200
Fetching venues for ... Pelham Parkway
Code: 200
Fetching venues for ... City Island
Code: 200
Fetching venues for ... Bedford Park
Code: 200
Fetching venues for ... University Heights
Code: 200
Fetching venues for ... Morris Heights
Code: 200
Fetching venues for ... Fordham
Code: 200
Fetching venues for ... East Tremont
Code: 200
Fetching venues for ... West Farms
Code: 200
Fetching venues for ... High  Bridge
Code: 200
Fetching venues for ... Melrose
Code: 200
Fetching venues for ... Mott Haven
Code

#### Checking contents of the resulting dataframe
First, size and columns

In [13]:
print(ny_venues.shape)
ny_venues.head()

(10258, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


Let's check also how many venues were returned for each neighborhood

In [14]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,29,29,29,29,29,29
Annadale,14,14,14,14,14,14
Arden Heights,4,4,4,4,4,4
Arlington,4,4,4,4,4,4
Arrochar,22,22,22,22,22,22
Arverne,19,19,19,19,19,19
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,21,21,21,21,21,21


#### Let's find out how many unique categories can be curated from all the returned venues

In [15]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 428 uniques categories.


## Analyze venues in each of the neighborhoods
#### First, classification of every venue entry

In [16]:
# Calculating a one-hot encoding
ny_venue_classes = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# Adding the neighborhood column back to dataframe
ny_venue_classes['Neighborhood'] = ny_venues['Neighborhood'] 

# Moving the neighborhood column to the first position
fixed_columns = [ny_venue_classes.columns[-1]] + list(ny_venue_classes.columns[:-1])
ny_venue_classes = ny_venue_classes[fixed_columns]

ny_venue_classes.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Prof

#### Now, grouping venue rows by neighborhood, calculating the mean of occurrence for each category

In [17]:
ny_classes_grouped = ny_venue_classes.groupby('Neighborhood').mean().reset_index()
ny_classes_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Prof

#### Let's reduce the classification
It means, considering only the most common venues in the neighborhood

First, defining a function to sort the venues in descending order for one neighborhood

In [18]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
 #   print (row_categories)
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now, creating a new dataframe, for only the top N venue classes for each neighborhood
* N will be fixed to 10 for this exercise

In [19]:
NUM_TOP_VENUES = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(NUM_TOP_VENUES):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ny_classes_sorted = pd.DataFrame(columns=columns)
ny_classes_sorted['Neighborhood'] = ny_classes_grouped['Neighborhood']

for ind in np.arange(ny_classes_grouped.shape[0]):
    ny_classes_sorted.iloc[ind, 1:] = most_common_venues(ny_classes_grouped.iloc[ind, :], NUM_TOP_VENUES)

ny_classes_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Chinese Restaurant,Supermarket,Bakery,Fast Food Restaurant,Electronics Store,Pharmacy,Martial Arts Dojo,Grocery Store
1,Annadale,Pizza Place,Liquor Store,American Restaurant,Pharmacy,Park,Diner,Restaurant,Food,Sports Bar,Dance Studio
2,Arden Heights,Pizza Place,Rental Car Location,Pharmacy,Coffee Shop,Women's Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
3,Arlington,Intersection,American Restaurant,Bus Stop,Grocery Store,Filipino Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant
4,Arrochar,Bus Stop,Deli / Bodega,Italian Restaurant,Pizza Place,Food Truck,Liquor Store,Sandwich Place,Athletics & Sports,Bagel Shop,Supermarket


## Preparing preferences information

#### First, fixing preferences
Note that this *prefs* dataframe encodes the preferences of the customer. It is associated directly to his/her profile, and quite stable

* Just a list of constant values for the example

In [20]:
params = pd.DataFrame({'PREFERENCES': [0.0, 1.0, 0.0,
                                 1.0, 0.0, 0.0, 1.0,
                                 0.0, 0.0, 0.0, 0.0, 0.0,
                                 1.0, 1.0]},                     
                index=['YOUNG', 'MATURE', 'OLD',
                       'SINGLE', 'MARRIED', 'CHILDREN', 'PETS',
                       'MUSIC', 'SPORTS', 'CULTURE', 'FASHION', 'GOING_OUT',
                       'HEALTH', 'RELIGIOUS'])

prefs = params.transpose()
prefs

,YOUNG,MATURE,OLD,SINGLE,MARRIED,CHILDREN,PETS,MUSIC,SPORTS,CULTURE,FASHION,GOING_OUT,HEALTH,RELIGIOUS
PREFERENCES,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


Note that is a mature single, wit pets, worried about healthy life and with religious interests

#### Loading a pre-configured file with preferences by customer segmentation
This *segments* dataframe encodes a classification for the different categories, according to people preferences

The values should come from specific surveys or history information. It must be always pre-calculated

* In this case, I have to provide a fixed-values file

In [21]:
customer_segments = pd.read_csv('NY_Categories.csv').set_index('CATEGORY')

# Translating NaN entries to zero 
customer_segments.fillna(0, inplace=True)

customer_segments.head()

,YOUNG,MATURE,OLD,SINGLE,MARRIED,CHILDREN,PETS,MUSIC,SPORTS,CULTURE,FASHION,GOING_OUT,HEALTH,RELIGIOUS
CATEGORY,,,,,,,,,,,,,,
Dessert Shop,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Pharmacy,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Ice Cream Shop,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Caribbean Restaurant,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
Donut Shop,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Now, mapping preferences to categories
It is just adding to the classification the preferences of the customer
* Values with a zero-score in customer preferences disable the column (also zero) in the segmentation dataframe
* Basically, it is just multiplication, but cell by cell

First, defining a function to perform such operation

In [22]:
def multiplyRow(row, pref_list):
    for i, item in enumerate(row):
        row[i] = item*pref_list[i]
    return row

Now, the multiplication function is called, using lambda functionality row by row (axis=1), with the customer preferences

In [23]:
# Using a copy, instead the loaded information. This is just because we are replacing values during operation
segments = customer_segments.copy()

full_prefs = segments.apply (lambda row: multiplyRow(row, prefs.iloc[0,:]), axis=1)

full_prefs.index.name = 'CATEGORY'
full_prefs.reset_index(inplace=True)

print('There are {} uniques categories.'.format(len(full_prefs['CATEGORY'].unique())))
full_prefs.head()

There are 428 uniques categories.


,CATEGORY,YOUNG,MATURE,OLD,SINGLE,MARRIED,CHILDREN,PETS,MUSIC,SPORTS,CULTURE,FASHION,GOING_OUT,HEALTH,RELIGIOUS
0,Dessert Shop,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Pharmacy,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Ice Cream Shop,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Caribbean Restaurant,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Donut Shop,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Calculating scores for neighborhoods
Finally, we will calculate a score for every neighborrhood, considering:
* The most common venue categories in it (calculated in the dataset ny_classes_sorted)
* The preferences from the customer, mapped to the categories (in the dataset full_prefs)

To get the neighborhood score, I am going to use the following algorithm:
- The list of categories is iterated, adding a weight according to its position (the most common, the higher number)
- A total multiplying factor is calculated for each of the categories, adding all individual flags in the preferences
- The total score is given by adding all the individual category scores

#### First, a function to calculate score for a single neighborhood
* An auxiliary function is used to calculate the multiplying factor of a category
* And the categories are weighted using position, according to previous NUM_TOP_VENUES

In [24]:
def getCategoryFactor(category, preferences):
    row = preferences.loc[preferences['CATEGORY'] == category]
    if (row.size == 0):
        return (0)
    return(row.iloc[0,1:].sum())

def getNeighborhoodScore(neighborhood, categories, preferences):
    total = 0
    for i, item in enumerate(categories):
        factor = getCategoryFactor(item, preferences)
        total = total + (factor*NUM_TOP_VENUES-i)
    #print('Score for neighborhood {} = {}'.format(neighborhood, total))
    return(total)

#### Scoring each of the neighborhoods
The function is called for each of the rows in the dataframe with the most common venues category for every neighborhoods

A new dataframe is created, with the calculated score and adding the coordinates for later presentation

In [25]:
scores_list=[]
    
for ind in np.arange(ny_classes_sorted.shape[0]):
    name = ny_classes_sorted.iloc[ind,0]
    
    # Calling the previously defined scoring funtion
    score = getNeighborhoodScore(name, ny_classes_sorted.iloc[ind,1:], full_prefs)

    scores_list.append([(name, score)])

scored_neighborhoods = pd.DataFrame([item for score_list in scores_list for item in score_list])
scored_neighborhoods.columns = ['Name','Score']
scored_neighborhoods.head()

,Name,Score
0,Allerton,175.0
1,Annadale,165.0
2,Arden Heights,95.0
3,Arlington,85.0
4,Arrochar,175.0


And the dataframe is sorted, according to the calculated score

The first 5 neighborhoods (with the highest score) are selected for presentation. Note that in case of same score, we keep the alphabetical order

In [26]:
scored_neighborhoods.sort_values(['Score'], axis = 0, ascending = False, inplace = True)
selected_neighborhoods = scored_neighborhoods.head(5)

selected_neighborhoods

,Name,Score
52,City Line,225.0
212,Pomonok,225.0
92,Erasmus,225.0
8,Auburndale,225.0
18,Beechhurst,215.0


## Creating a map with resulting neighborhoods

#### Adding coordinates to the selected neighborhoods

In [27]:
selected_geo_list=[]

for ind in np.arange(selected_neighborhoods.shape[0]):
    name = selected_neighborhoods.iloc[ind,0]
    score = selected_neighborhoods.iloc[ind,1]
    
    # The coordinates are fetched for the original neighborhoods dataframe, looking for the name
    latitude = neighborhoods.loc[neighborhoods['Neighborhood'] == name, ['Latitude']].values[0,0]
    longitude = neighborhoods.loc[neighborhoods['Neighborhood'] == name, ['Longitude']].values[0,0]
    
    selected_geo_list.append([(name, score, latitude, longitude)])

selected_coordinates = pd.DataFrame([item for selected_item in selected_geo_list for item in selected_item])
selected_coordinates.columns = ['Name','Score','Latitude','Longitude']
selected_coordinates

,Name,Score,Latitude,Longitude
0,City Line,225.0,40.678570,-73.867976
1,Pomonok,225.0,40.734936,-73.804861
2,Erasmus,225.0,40.646926,-73.948177
3,Auburndale,225.0,40.761730,-73.791762
4,Beechhurst,215.0,40.792781,-73.804365


#### Use geopy library to get the latitude and longitude values of New York City

To define the geocoder instance, the user_agent will be <em>ny_explorer</em>

In [28]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Creating a New York map with selected neighborhoods on it
The labels include the position in the selection list and the calculated score

In [29]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add markers to map
for i, lat, lng, name, score in zip(selected_coordinates.index, selected_coordinates['Latitude'], selected_coordinates['Longitude'], selected_coordinates['Name'], selected_coordinates['Score']):
    
    # The popup labels will show the position (index+1) and the score
    label = '[{}] {} (Score: {})'.format(i+1, name, score)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)

map_newyork

<a id='item5'></a>